In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn import preprocessing

In [3]:
df = pd.read_excel("/content/drive/MyDrive/Train_labled.xlsx") # указываем путь до Train_labled
test = pd.read_excel("/content/drive/MyDrive/Test_unlabled.xlsx") # указываем путь до Test_unlabled

In [4]:
def process_df(original_df):
  df = original_df.copy()

  # df = df.drop(['рN', 'M','K+ max', 'эффект'], axis = 1) # удаляем неинформативные колонки
  df.loc[(~np.isnan(df['вид рецидива'])), 'Рецидив'] = 1 # если лечили, но рецидива нет, то рецидив есть

  recedive_cols = ['вид рецидива', 'мтс', 'местный', 'Лечение рецидива',
          'системное лечение', 'эффект лечения рецидива']
  df.loc[df['Рецидив'] == 0, recedive_cols] = -1 #заполняем nan в столбцах рецидива
  # df = df.drop(recedive_cols, axis = 1)
  df['Время операции'] = df['Время операции'].fillna(0) # если операции не было, то время операции = 0
  return df

## Проверяем ещё раз accuracy на train_labled

In [5]:
def make_heuristic_predict(data):
  predictions = pd.DataFrame([0] * len(data), columns = ['Predictions'])
  predictions.loc[(data['Рецидив'] == 1) & (data['диурез, сутки'] >= 2000), 'Predictions'] = 1
  predictions.loc[(data['Рецидив'] == 1) & np.isnan(df['диурез, сутки']), 'Predictions'] = 2
  return predictions['Predictions']

In [6]:
processed_df = process_df(df)
targets_processed_df = processed_df['Текущий прогноз']

In [7]:
balanced_accuracy_score(make_heuristic_predict(processed_df), targets_processed_df)

0.9813373253493015

## Делаем предикт на test_unlabled

In [8]:
processed_test = process_df(test)

In [9]:
test_predictions = make_heuristic_predict(processed_test).values
test_predictions

array([2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0])

In [10]:
test['Текущий прогноз'] = test_predictions

In [11]:
test.to_csv('Result.csv')